In [1]:
# only adjust settings in this cell
state = 'AL'
root_name = 'Jefferson'# which county will root our district (or county_cluster==multi-district)?
k = 7                  # the state has this number of districts
deviation = 1          # use: L=ideal_population-deviation, U=ideal_population-deviation
size = 2               # size=1 for single district, size=2 for double district, ...
time_limit = 48*3600   # time limit in seconds

In [2]:
filepath = 'C:\\districting-data-2020\\'
filename = state + '_county.json'

from cluster import read_graph_from_json
G = read_graph_from_json( filepath + filename) 

In [3]:
G._k = k
G._size = size
G._time_limit = time_limit

print("State has total of k =",G._k,"districts")
print("We seek a multi-district (county cluster) of size =",G._size)

State has total of k = 7 districts
We seek a multi-district (county cluster) of size = 2


In [4]:
ideal_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes ) / G._k

# Calculate lower and upper population limits
from math import ceil, floor
G._L = ceil( ideal_population - deviation )
G._U = floor( ideal_population + deviation )

print("Single district lower population limit L =",G._L)
print("Single district upper population limit U =",G._U)

Single district lower population limit L = 717754
Single district upper population limit U = 717755


In [5]:
# Require this county be in our district/county_cluster/multi_district
G._root = G._root = [ i for i in G.nodes if G.nodes[i]['NAME20'] == root_name ][0]
print("Use",root_name,"County to be the root. In our graph, this is vertex #",G._root)

Use Jefferson County to be the root. In our graph, this is vertex # 22


In [ ]:
import simple_enumerator
districts = simple_enumerator.simple_enumerator(G)

Sol# Node# Time(s) district
1 15608 0.12 [22, 45, 0, 64, 14, 26, 43, 36, 33, 56, 54, 39]
2 37976 0.3 [22, 45, 0, 64, 14, 43, 54, 39, 33, 46, 51, 40]
3 143594 1.22 [22, 45, 0, 64, 58, 50, 26, 43, 52, 54]
4 146638 1.25 [22, 45, 0, 64, 58, 50, 2, 36, 17, 56, 54, 39]
5 238484 2.14 [22, 45, 0, 64, 58, 26, 43, 36, 33, 46, 40, 13, 20, 53]
6 281860 2.53 [22, 45, 0, 64, 58, 26, 32, 39, 33, 1, 9, 56]
7 319564 2.9 [22, 45, 0, 64, 58, 2, 43, 17, 56, 65, 9, 46, 40, 53]
8 403960 3.69 [22, 45, 0, 64, 58, 43, 52, 36, 33, 32, 17, 46, 54, 37]
9 443019 4.04 [22, 45, 0, 64, 58, 43, 36, 33, 32, 65, 46, 40, 20, 54, 37]
10 742840 7.06 [22, 45, 0, 64, 23, 52, 2, 43, 36, 33, 9, 17, 46, 39]
11 746505 7.1 [22, 45, 0, 64, 23, 52, 2, 43, 17, 39, 33, 46, 40, 53, 37]
12 895149 8.59 [22, 45, 0, 64, 23, 57, 2, 43, 17, 54, 32, 39, 33, 46, 40]
13 899367 8.62 [22, 45, 0, 64, 23, 57, 2, 43, 39, 33, 46, 51, 40, 53, 37]
14 914460 8.78 [22, 45, 0, 64, 23, 57, 2, 56, 33, 9, 51, 40, 53, 39]
15 933764 8.97 [22, 45, 0, 64, 23, 5

In [ ]:
import math

def objective(DG, district, obj_type):
    district_bool = { i : False for i in DG.nodes }
    for i in district:
        district_bool[i] = True
    if obj_type == 'cut_edges':
        return sum( 1 for u in district for v in DG.neighbors(u) if not district_bool[v] )
    elif obj_type == 'perimeter':
        internal_perim = sum( DG.edges[u,v]['shared_perim'] for u in district for v in DG.neighbors(u) if not district_bool[v] )
        external_perim = sum( DG.nodes[i]['boundary_perim'] for i in district if DG.nodes[i]['boundary_node'] ) 
        return internal_perim + external_perim
    elif obj_type =='inverse_polsby_popper':
        internal_perim = sum( DG.edges[u,v]['shared_perim'] for u in district for v in DG.neighbors(u) if not district_bool[v] )
        external_perim = sum( DG.nodes[i]['boundary_perim'] for i in district if DG.nodes[i]['boundary_node'] ) 
        P = internal_perim + external_perim
        A = sum( DG.nodes[i]['area'] for i in district )
        return P * P / ( 4 * math.pi * A )
    else:
        assert False

In [ ]:
# Draw the districting plans.
filename = state + '_county.shp'
from cluster import draw_single_district

In [ ]:
obj_type = 'cut_edges'
min_obj = min( objective(G, districts[p], obj_type) for p in range(len(districts)) )
print("For objective type",obj_type,"here are the identified maps within 10% of the incumbent objective =",min_obj)
for p in range(len(districts)):
    if objective(G, districts[p], obj_type) <= 1.10 * min_obj:
        draw_single_district( filepath, filename, G, districts[p], zoom=False )

In [ ]:
obj_type = 'perimeter'
min_obj = min( objective(G, districts[p], obj_type) for p in range(len(districts)) )
print("For objective type",obj_type,"here are the identified maps within 10% of the incumbent objective =",min_obj)
for p in range(len(districts)):
    if objective(G, districts[p], obj_type) <= 1.10 * min_obj:
        draw_single_district( filepath, filename, G, districts[p], zoom=False )

In [ ]:
obj_type = 'inverse_polsby_popper'
min_obj = min( objective(G, districts[p], obj_type) for p in range(len(districts)) )
print("For objective type",obj_type,"here are the identified maps within 10% of the incumbent objective =",min_obj)
for p in range(len(districts)):
    if objective(G, districts[p], obj_type) <= 1.10 * min_obj:
        draw_single_district( filepath, filename, G, districts[p], zoom=False )